### Mini-project 4: Caloric intake vs burned tracker
**Goal:**
Track each crew member's daily caloric surplus/deficit, then:
- Flag energy imbalance (e.g., extended deficit): flag any day where deficit >500 kcal
- Flag crew members with >5 deficit days total
- Summarize overall gain/loss over time
- Plot balance trends over the mission

In [1]:
import pandas as pd

In [2]:
caloric = pd.read_csv('./data/caloric_balance_log.csv')

In [3]:
print(caloric)

          date   crew_member  calories_in  calories_out
0   2025-03-20  Alice Carter         2482          2848
1   2025-03-20    Ben O'Hara         2567          2981
2   2025-03-20  Jonas Müller         2367          2907
3   2025-03-20    Nina Patel         2108          2993
4   2025-03-20     Leo Zhang         2595          2390
..         ...           ...          ...           ...
79  2025-04-02    Ben O'Hara         2229          2685
80  2025-04-02  Jonas Müller         1994          2884
81  2025-04-02    Nina Patel         2493          2843
82  2025-04-02     Leo Zhang         2139          2902
83  2025-04-02    Emma López         2149          2780

[84 rows x 4 columns]


In [6]:
crew = caloric['crew_member'].unique()
crew

array(['Alice Carter', "Ben O'Hara", 'Jonas Müller', 'Nina Patel',
       'Leo Zhang', 'Emma López'], dtype=object)

In [8]:
cal_diff = []
for index, row in caloric.iterrows():
    diff = row['calories_in'] - row['calories_out']
    cal_diff.append(diff)

caloric['cal_diff'] = cal_diff

In [9]:
caloric.head()

,date,crew_member,calories_in,calories_out,cal_diff
0,2025-03-20,Alice Carter,2482,2848,-366
1,2025-03-20,Ben O'Hara,2567,2981,-414
2,2025-03-20,Jonas Müller,2367,2907,-540
3,2025-03-20,Nina Patel,2108,2993,-885
4,2025-03-20,Leo Zhang,2595,2390,205


In [14]:
status = []
for index, row in caloric.iterrows():
    diff = row['cal_diff']
    if diff < 0:
        stat = 'deficit'
    elif diff == 0:
        stat = 'balanced'
    else:
        stat = 'surplus'
    status.append(stat)